In [11]:
import os
import sys
import torch
import pandas as pd
import numpy as np
from torch.distributions import Normal, MultivariateNormal

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from deep_fields.models.random_fields.poisson_covariance import MertonBirthPoissonCovariance

In [12]:
data_dir = "C:/Users/cesar/Desktop/Projects/General/deep_random_fields/data/raw/merton_birth_covariance/"
my_data_path = os.path.join(data_dir, "merton_birth_simulation.tr")
data_ = torch.load(my_data_path)
data_loader = data_["data_loader"]
model_param = data_["model_param"]
inference_param = MertonBirthPoissonCovariance.get_inference_parameters()
inference_param.update({"nmc": 3000,
                        "burning": 1500,
                        "metrics_logs": 50})

In [13]:
mbpc = MertonBirthPoissonCovariance(None, data_loader, None, **model_param)
monte_carlo_values = mbpc.initialize_inference(data_loader,**inference_param)

New Model Set For Inference
Data Loader has No Parameters


In [19]:
location_index = 0
locations_sample_now = monte_carlo_values["locations_history"]#
K = monte_carlo_values["K"]#
kernel = monte_carlo_values["kernel"]#
interest_rate = data_loader["expected_returns_history"]
log_returns = data_loader["log_returns"]

locations = mbpc.locations_sample_to_tensor(locations_sample_now)#
location_now = locations_sample_now[location_index][-1]#
location_proposal = mbpc.locations_prior.sample().detach().numpy()#

In [20]:
new_location = location_proposal

In [21]:
mbpc.total_assets_in_history

tensor(119)

In [22]:
if isinstance(locations, np.ndarray):
    locations = torch.Tensor(locations)
if isinstance(new_location,np.ndarray):
    new_location = torch.Tensor(new_location)

index_left = list(range(mbpc.total_assets_in_history.item()))
index_left.remove(location_index)

new_location = torch.Tensor(new_location).unsqueeze(0)
K_new = new_kernel(locations, location_index, new_location, mbpc.total_assets_in_history.item(), K, kernel)

In [23]:
likelihood_mean = interest_rate[None, :] 
realizations_pdf_new = 

In [25]:
likelihood_mean.shape

torch.Size([1, 119])

In [61]:
data_loader.keys()

dict_keys(['birth_numbers', 'assets_in_the_market', 'total_assets_in_history', 'log_returns', 'locations_history', 'kernel_sigma', 'kernel_lenght_scales', 'covariance_diffusion_history', 'expected_returns_history', 'kernel'])

In [59]:
def birth_log_likelihood_realizations(new_location, locations, location_index, data_loader, monte_carlo_values):
    """
    
    """
    current_log_probability = 0.
    number_of_realizations = log_returns.shape[0]
    for time_index in range(number_of_realizations):
        current_assets_in_the_market = mbpc.assets_in_the_market[time_index]
        current_interest_rate = interest_rate[None, :current_assets_in_the_market]
        current_diffusion_covariance = K_new[:current_assets_in_the_market,:current_assets_in_the_market]
        current_diffusion_distribution = MultivariateNormal(current_interest_rate, torch.Tensor(current_diffusion_covariance))
        current_log_returns = log_returns[time_index,:current_assets_in_the_market]

        current_log_probability += current_diffusion_distribution.log_prob(current_log_returns[None,:])
    return current_log_probability

tensor([-564.9484])
tensor([-1353.0200])
tensor([-1296.5092])
tensor([-1292.9525])
tensor([-1415.0032])


In [41]:
log_returns.shape

torch.Size([5, 119])

In [32]:
realizations_pdf_new.log_prob(log_returns)

tensor([-10736.1172, -49016.5469, -30262.4453,  -5772.4102,    -85.3053])

In [18]:
def new_kernel(locations, location_index, new_location, number_of_arrivals, K, kernel):
    if isinstance(locations, np.ndarray):
        locations = torch.Tensor(locations)
    if isinstance(new_location,np.ndarray):
        new_location = torch.Tensor(new_location)

    index_left = list(range(number_of_arrivals))
    index_left.remove(location_index)

    k_new = K[location_index, :].copy()
    K_new = K.copy()

    new_locations = torch.cat([locations[0:location_index], locations[location_index + 1:]])

    k01 = kernel(new_location, new_locations).evaluate().detach().numpy()
    k00 = kernel(new_location, new_location).evaluate().detach().numpy()
    k_new[index_left] = k01
    k_new[location_index] = k00

    K_new[location_index, :] = k_new.copy()
    K_new[:, location_index] = k_new.T.copy()

    return K_new

In [ ]:
def log_likelihood_realizations(self, new_location, locations, location_index, data_loader, monte_carlo_values):
    K = monte_carlo_values["K"]
    kernel = monte_carlo_values["kernel"]

    interest_rate = data_loader["expected_returns_history"]
    log_returns = data_loader["log_returns"]

    new_location = torch.Tensor(new_location).unsqueeze(0)
    K_new = new_kernel(locations, location_index, new_location, self.number_of_processes, K, kernel)

    likelihood_mean = interest_rate[None, :] 
    realizations_pdf_new = MultivariateNormal(likelihood_mean, torch.Tensor(K_new))

    return realizations_pdf_new.log_prob(log_returns).item()

In [ ]:
def gibbs_locations(self, data_loader, location_index, monte_carlo_values):
    locations_sample_now = monte_carlo_values["locations"]#
    K = monte_carlo_values["K"]#
    kernel = monte_carlo_values["kernel"]#

    locations = self.locations_sample_to_tensor(locations_sample_now)#
    location_now = locations_sample_now[location_index][-1]#
    location_proposal = self.locations_prior.sample().detach().numpy()#

    new_location, ll = elliptical_slice(initial_theta=location_now,
                                        prior=location_proposal,
                                        lnpdf=self.log_likelihood_realizations,
                                        pdf_params=(locations, location_index, data_loader, monte_carlo_values))

    K_new = new_kernel(locations, location_index, torch.Tensor(new_location).unsqueeze(0), self.number_of_processes,
                       K, kernel)
    
    monte_carlo_values["locations"][location_index].append(new_location)
    monte_carlo_values["K"] = K_new

    return monte_carlo_values

In [ ]:
covariance_diffusion_history